In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN


smoten_median_imputed_less_40 = pd.read_csv('smoten_median_imputed_less_40.csv', index_col=False)

pd.set_option("display.max_columns", None)


def ensembleScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('feature_names_in_')
    print(clf.feature_names_in_)
    print()
    print('feature_importances_')
    print(clf[1].feature_importances_)
    print()
    for i in range(len(clf.feature_names_in_)):
        print(clf.feature_names_in_[i], ":" ,clf[1].feature_importances_[i])
    print()
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print()
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print()
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print()
    print('\n')
    

print('AdaBoost Ensemble')

addidas = AdaBoostClassifier(estimator=None, n_estimators=50, 
                             learning_rate=0.201, algorithm='SAMME.R', 
                             random_state=None)



print('### SMOTEN median impute ###')
print()

X = smoten_median_imputed_less_40.loc[:, ~smoten_median_imputed_less_40.columns.isin(['heart_rate_max', 'mbp_min', 'mbp_mean', 'sbp_max', 'dbp_max', 'glucose_max', 'wbc_min', 'creatinine_max', 'pt_min', 'sofa_cardiovascular', 'charlson_comorbidity_index','outcome'])]
y = smoten_median_imputed_less_40['outcome']

print('StandardScaler')
print()
ensembleScale_coef(StandardScaler(), addidas, 5, X, y)

print('RobustScaler')
print()
X = smoten_median_imputed_less_40.loc[:, ~smoten_median_imputed_less_40.columns.isin(['gender', 'mbp_min', 'mbp_max', 'mbp_mean', 'sbp_min', 'temperature_min', 'temperature_max', 'glucose_min', 'wbc_max', 'hemoglobin_max', 'outcome'])]
y = smoten_median_imputed_less_40['outcome']
ensembleScale_coef(RobustScaler(), addidas, 5, X, y)


AdaBoost Ensemble
### SMOTEN median impute ###

StandardScaler

accuracy of each fold - [0.9633105802047781]
Avg accuracy : 0.19266211604095562
score 
 0.9615863196814562
feature_names_in_
['gender' 'age' 'heart_rate_min' 'heart_rate_mean' 'mbp_max' 'sbp_min'
 'sbp_mean' 'dbp_min' 'dbp_mean' 'temperature_min' 'temperature_max'
 'temperature_mean' 'glucose_min' 'wbc_max' 'creatinine_min'
 'hemoglobin_min' 'hemoglobin_max' 'pt_max' 'urineoutput'
 'sofa_coagulation' 'sofa_cns' 'sofa_renal']

feature_importances_
[0.   0.14 0.   0.06 0.   0.02 0.04 0.02 0.06 0.   0.   0.08 0.02 0.04
 0.04 0.04 0.04 0.08 0.12 0.02 0.12 0.06]

gender : 0.0
age : 0.14
heart_rate_min : 0.0
heart_rate_mean : 0.06
mbp_max : 0.0
sbp_min : 0.02
sbp_mean : 0.04
dbp_min : 0.02
dbp_mean : 0.06
temperature_min : 0.0
temperature_max : 0.0
temperature_mean : 0.08
glucose_min : 0.02
wbc_max : 0.04
creatinine_min : 0.04
hemoglobin_min : 0.04
hemoglobin_max : 0.04
pt_max : 0.08
urineoutput : 0.12
sofa_coagulation : 0.02
so